In [2]:
import pandas as pd

In [6]:
raw_data = pd.read_csv('data/train_data.csv')
raw_data

,id,content,annotation
0,5ad67f86,出院记录\n入院情况/主诉：左上肢麻木伴发作性言语不清5天”入院。\n入院情况/辅助检查：。...,T48\tE95f2a617 0 4\t出院记录\nT49\tE95f2a617 5 9\t...
1,345668a2,入院病历\n暂缺\n第一胎第一产，足月顺产，按时预防接种，混合喂养，有添加辅食，三个月抬头，...,T79\tE95f2a617 0 4\t入院病历\nT80\tE1ceb2bd7 5 7\t...
2,62b0277d,入院病历\n反复右下肢一过性痉挛2小时\n患者于今日中午吃完饭后无明显诱因出现右下肢一过性痉...,T53\tE95f2a617 0 4\t入院病历\nT1\tE3d0cb547 5 7\t反...
3,1a0478f4,"出院记录\n1、患者赵孙超，男,30岁，因“车祸致右下肢肿胀、疼痛，伴活动受限12小时”入院...",T49\tE95f2a617 0 4\t出院记录\nT40\tE8ff29ca5 9 12\...
4,e04c5425,入院病历\n心前区无隆起，未及震颤，心界不大，心律不齐，心率60次/分，未闻及杂音。\n生于...,T68\tE95f2a617 0 4\t入院病历\nT69\tE320ca3f6 5 8\t...
...,...,...,...
1890,a28143fc,"入院记录\n主诉:\n发热、咳嗽3天\n现病史:\n入院前3天,患儿无明显诱因下出现发热,体...",T620\tE95f2a617 0 4\t入院记录\nT621\tE95f2a617 5 7...
1891,df4fd86a,"首程录\n病例特点:\n1、患者女,3岁3个月。2、患者因“发热、咳嗽3天”入院。3、查体:...",T209\tE95f2a617 0 3\t首程录\nT210\tE95f2a617 4 8\...
1892,3e22c51,"出院小结\n入院诊断:\n社区获得性肺炎,非重症\n出院日期:\n2016年12月11日\n...",T895\tE95f2a617 0 4\t出院小结\nT896\tE95f2a617 5 9...
1893,5b2c266,...,T185\tE95f2a617 105 110\t病史陈述者\nT186\tE8df2997...


In [47]:
def takename(elem):
    return int(elem.split()[0])


raw_data = raw_data.dropna(axis=0).reset_index(drop=True)
parsed_data = pd.DataFrame(columns=['id', 'dependency'])
for index, line in raw_data.iterrows():
    annotation = line['annotation']
    anno_list = annotation.split('\n')
    anno_list = list(filter(None, anno_list))
    #entity_dict: {'T48': ['E95f2a617', '0', '4', '出院记录'], 'T49': ['E95f2a617', '5', '9', '入院情况']}
    entity_dict = {}
    entry = list()
    for anno in anno_list:
        if anno[0] == 'T':
            anno = anno.split()
            if len(anno) is not 5:
                break
            entity_dict[anno[0]] = anno[1:]
        elif anno[0] == 'R':
            anno = anno.split()
            if len(anno) is not 4:
                break
            #anno: [R1, R742a31d5, Arg1:T1, Arg2:T2]
            father_node = anno[2].split(':')[1]
            child_node = anno[3].split(':')[1]
            relation = anno[1]
            entry.append(child_node[1:] + '\t' + entity_dict[child_node][0] + '\t' + entity_dict[child_node][3] + '\t' +\
                        father_node[1:] + '\t' + entity_dict[father_node][0] + '\t' + entity_dict[father_node][3] + '\t' +\
                         relation)
    entry.sort(key=takename)
    entry_str = ''
    for e in entry:
        entry_str = entry_str + '\n' + e
    parsed_data.loc[len(parsed_data)] = [line['id'], entry_str]
parsed_data
parsed_data.to_csv('data/data.csv', encoding='utf-8')            

In [ ]:
"""
dependency example for each text:

child-node	child-type	child-text	father-node 	father-type     	 relation
        2	E340ca71c	麻木   	         1  	 E320ca3f6 	左上肢 	R742a31d5
        4	E340ca71c	言语不清	       3    	E1ceb2bd7	发作性 	R742a31d5
        4	E340ca71c	言语不清	       5    	E1deb2d6a	5天  	R742a31d5
        9	E310ca263	改善循环	       8    	E1deb2d6a	入院后 	R742a31d5
        ......
"""